In [1]:
import pandas as pd
from time import time
import os

# Read & Preprocess

In [2]:
from preprocess import read_worklogs_files, preprocess_worklogs

In [3]:
t = time()
worklogs = read_worklogs_files()
print(time() - t)

122.41984677314758


In [4]:
t = time()
worklogs = preprocess_worklogs(worklogs)
print(time() - t)

4.0200114250183105


In [5]:
pd.to_pickle(worklogs, '../data/worklogs.pkl')

# Calculate

In [6]:
from calculate import calculate_worklogs, calculate_users_interval, calculate_metrics, calculate_performance

In [7]:
t = time()
worklogs_calc, issues, projects, info = calculate_worklogs(worklogs)
print(time() - t)

------- Worklogs -------
min_date     2019-01-01
max_date     2023-01-14
#logs             83396
#issues           19667
#projects            49
#users               51
dtype: object
9.143399238586426


In [ ]:
t = time()

intervals = {
    '2019': ['20190101', '20191231'],
    '2020': ['20200101', '20201231'],
    '2021': ['20210101', '20211231'],
    '2022': ['20220101', '20221231'],
    '2019-2022': ['20190101', '20221231'], 
}

users_and_metrics = {}
for key, interval in intervals.items():
    users_calc = calculate_users_interval(worklogs_calc, interval)
    metrics = calculate_metrics(users_calc)
    performance = calculate_performance(metrics)
    data = users_calc.join(metrics).join(performance, rsuffix='_std')
    users_and_metrics[key] = data
print(time() - t)

In [ ]:
pd.to_pickle(worklogs_calc, '../data/worklogs_calc.pkl')
pd.to_pickle(users_and_metrics, '../data/users_and_metrics.pkl')

# Report

In [ ]:
from conventions import Report
from report import data_to_plot, employee_report

In [ ]:
year = 2022
history = range(2019, 2022)
employees, info, kpi, score, yearly, time_split, by_project = data_to_plot(users_and_metrics, year, history)

In [ ]:
t = time()
for employee in employees:
    employee_report(employee, 
                    year, 
                    info.loc[employee], 
                    kpi.loc[employee], 
                    score, 
                    time_split.loc[employee],
                    by_project.loc[employee], 
                    yearly.loc[employee], 
                    save=Report.IMAGES_FOLDER + employee, 
                    dpi=1000)
print(time() - t)   